# Lab 1: Kubernetes Fundamentals

## 🎯 Objectives
- Hiểu Kubernetes architecture và core concepts
- Setup local Kubernetes cluster (minikube/kind)
- Sử dụng kubectl CLI
- Làm quen với namespaces

## 📋 Prerequisites
- Docker đã được cài đặt
- kubectl đã được cài đặt
- minikube hoặc kind đã được cài đặt

## 🚀 Quick Start với WSL + Minikube

Nếu bạn đang dùng WSL với minikube đã cài sẵn:

1. **Mở WSL terminal** và start minikube:
```bash
minikube start
```

2. **Chạy notebook này trong VS Code** với Python kernel từ WSL

3. Hoặc chạy trực tiếp các lệnh trong WSL terminal

In [2]:
# Bước 1: Setup để chạy lệnh qua WSL từ Windows
# Cell này sẽ tự động gọi kubectl thông qua WSL

import subprocess
import sys

# Kiểm tra xem đang chạy trên Windows hay không
IS_WINDOWS = sys.platform == 'win32'

def run_wsl(cmd):
    """Run command through WSL from Windows"""
    if IS_WINDOWS:
        # Chạy lệnh qua WSL
        full_cmd = f'wsl {cmd}'
    else:
        full_cmd = cmd
    
    try:
        result = subprocess.run(
            full_cmd,
            shell=True,
            capture_output=True,
            text=True,
            timeout=300
        )
        output = result.stdout + result.stderr
        return output if output.strip() else "No output"
    except subprocess.TimeoutExpired:
        return "Error: Command timed out"
    except Exception as e:
        return f"Error: {str(e)}"

print(f"🖥️ Running on: {'Windows (will use WSL)' if IS_WINDOWS else 'Linux/Mac'}")
print("\n🔍 Checking minikube status...")
print(run_wsl("minikube status"))

🖥️ Running on: Windows (will use WSL)

🔍 Checking minikube status...
minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured


minikube
type: Control Plane
host: Running
kubelet: Running
apiserver: Running
kubeconfig: Configured




In [3]:
# Bước 2: Start minikube nếu chưa chạy
# Chỉ cần chạy cell này nếu minikube chưa start

print("🚀 Starting minikube (có thể mất 1-2 phút)...")
print(run_wsl("minikube start"))
print("\n✅ Done! Kiểm tra lại status:")
print(run_wsl("minikube status"))

🚀 Starting minikube (có thể mất 1-2 phút)...
* minikube v1.37.0 on Ubuntu 24.04 (kvm/amd64)
* Using the docker driver based on existing profile
* Starting "minikube" primary control-plane node in "minikube" cluster
* Pulling base image v0.0.48 ...
* Verifying Kubernetes components...
  - Using image gcr.io/k8s-minikube/storage-provisioner:v5
* Enabled addons: storage-provisioner, default-storageclass
* Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


✅ Done! Kiểm tra lại status:
* minikube v1.37.0 on Ubuntu 24.04 (kvm/amd64)
* Using the docker driver based on existing profile
* Starting "minikube" primary control-plane node in "minikube" cluster
* Pulling base image v0.0.48 ...
* Verifying Kubernetes components...
  - Using image gcr.io/k8s-minikube/storage-provisioner:v5
* Enabled addons: storage-provisioner, default-storageclass
* Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


✅ Done! Kiểm t

## 1. Kubernetes Architecture

Kubernetes là một container orchestration platform với kiến trúc phân tán:

```
┌─────────────────────────────────────────┐
│         Control Plane (Master)         │
│  ┌──────────┐  ┌──────────┐           │
│  │ API      │  │ etcd     │           │
│  │ Server   │  │ (State)  │           │
│  └──────────┘  └──────────┘           │
│  ┌──────────┐  ┌──────────┐           │
│  │Scheduler │  │Controller│           │
│  │          │  │ Manager  │           │
│  └──────────┘  └──────────┘           │
└─────────────────────────────────────────┘
              │
    ┌─────────┴─────────┐
    │                   │
┌───▼────┐         ┌───▼────┐
│ Node 1 │         │ Node 2 │
│ ┌────┐ │         │ ┌────┐ │
│ │Pod │ │         │ │Pod │ │
│ └────┘ │         │ └────┘ │
└────────┘         └────────┘
```

### Core Components:
- **Control Plane**: Quản lý cluster
- **Nodes**: Worker machines chạy containers
- **Pods**: Smallest deployable unit
- **kubectl**: CLI tool để interact với cluster


In [4]:
def run_kubectl(command):
    """Run kubectl command through WSL"""
    # Gọi kubectl thông qua WSL
    return run_wsl(command)

# Check kubectl version
print("🔍 Checking kubectl version:")
print(run_kubectl("kubectl version --client"))

🔍 Checking kubectl version:
Client Version: v1.34.1
Kustomize Version: v5.7.1

Client Version: v1.34.1
Kustomize Version: v5.7.1



In [5]:
# Check cluster info
print("📊 Cluster Information:")
print(run_kubectl("kubectl cluster-info"))

print("\n📋 Nodes:")
print(run_kubectl("kubectl get nodes"))


📊 Cluster Information:
Kubernetes control plane is running at https://127.0.0.1:32771
CoreDNS is running at https://127.0.0.1:32771/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


📋 Nodes:
NAME       STATUS   ROLES           AGE   VERSION
minikube   Ready    control-plane   8h    v1.34.0

Kubernetes control plane is running at https://127.0.0.1:32771
CoreDNS is running at https://127.0.0.1:32771/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


📋 Nodes:
NAME       STATUS   ROLES           AGE   VERSION
minikube   Ready    control-plane   8h    v1.34.0



## 2. Namespaces

Namespaces giúp tổ chức và isolate resources trong cluster.

Trong Kubernetes, namespace là không gian phân tách logic dùng để tổ chức và cô lập tài nguyên trong cluster.

Nói dễ hiểu:
👉 Namespace giống như thư mục trong hệ thống file, còn tài nguyên Kubernetes (pod, service, deployment, configmap…) giống như các file.

🧩 Namespace dùng để làm gì?
1. Tách biệt tài nguyên

- Team A dùng namespace team-a

- Team B dùng namespace team-b
→ Không ảnh hưởng lẫn nhau.

2. Quản lý phân quyền (RBAC)

Bạn có thể giới hạn user chỉ xem/tạo tài nguyên trong namespace của họ.

3. Giới hạn tài nguyên (Resource Quota)

Mỗi namespace có thể được giới hạn: CPU, RAM ,Số lượng pod, Storage

4. Quản lý môi trường

Tách môi trường: dev, staging, production

Mỗi môi trường là một namespace riêng trong cùng 1 cluster.

### Default Namespaces:
- `default`: Default namespace cho resources
- `kube-system`: System components
- `kube-public`: Public resources
- `kube-node-lease`: Node heartbeat


In [6]:
# List namespaces
print("📁 Namespaces:")
print(run_kubectl("kubectl get namespaces"))

# Create a new namespace
print("\n➕ Creating lab namespace:")
print(run_kubectl("kubectl create namespace k8s-lab --dry-run=client -o yaml"))
print("\n💡 To create: kubectl create namespace k8s-lab")


📁 Namespaces:
NAME              STATUS   AGE
default           Active   8h
kube-node-lease   Active   8h
kube-public       Active   8h
kube-system       Active   8h


➕ Creating lab namespace:
apiVersion: v1
kind: Namespace
metadata:
  name: k8s-lab
spec: {}
status: {}


💡 To create: kubectl create namespace k8s-lab
NAME              STATUS   AGE
default           Active   8h
kube-node-lease   Active   8h
kube-public       Active   8h
kube-system       Active   8h


➕ Creating lab namespace:
apiVersion: v1
kind: Namespace
metadata:
  name: k8s-lab
spec: {}
status: {}


💡 To create: kubectl create namespace k8s-lab


## 3. kubectl Basics

### Common Commands:
- `kubectl get <resource>`: List resources
- `kubectl describe <resource> <name>`: Describe resource
- `kubectl create`: Create resource
- `kubectl apply -f <file>`: Apply manifest file
- `kubectl delete`: Delete resource
- `kubectl logs <pod>`: View logs
- `kubectl exec -it <pod> -- <command>`: Execute command in pod


In [7]:
# Explore kubectl commands
print("📚 Available API Resources:")
resources = run_kubectl("kubectl api-resources --verbs=list --namespaced -o name")
print(resources[:500] + "..." if len(resources) > 500 else resources)

print("\n💡 Key Resources:")
print("  - pods (po)")
print("  - deployments (deploy)")
print("  - services (svc)")
print("  - configmaps (cm)")
print("  - secrets")
print("  - persistentvolumes (pv)")


📚 Available API Resources:
configmaps
endpoints
events
limitranges
persistentvolumeclaims
pods
podtemplates
replicationcontrollers
resourcequotas
secrets
serviceaccounts
services
controllerrevisions.apps
daemonsets.apps
deployments.apps
replicasets.apps
statefulsets.apps
horizontalpodautoscalers.autoscaling
cronjobs.batch
jobs.batch
leases.coordination.k8s.io
endpointslices.discovery.k8s.io
events.events.k8s.io
ingresses.networking.k8s.io
networkpolicies.networking.k8s.io
poddisruptionbudgets.policy
rolebindings.rbac.auth...

💡 Key Resources:
  - pods (po)
  - deployments (deploy)
  - services (svc)
  - configmaps (cm)
  - secrets
  - persistentvolumes (pv)
configmaps
endpoints
events
limitranges
persistentvolumeclaims
pods
podtemplates
replicationcontrollers
resourcequotas
secrets
serviceaccounts
services
controllerrevisions.apps
daemonsets.apps
deployments.apps
replicasets.apps
statefulsets.apps
horizontalpodautoscalers.autoscaling
cronjobs.batch
jobs.batch
leases.coordination.k8s.io

## 4. Tóm tắt và Next Steps

### ✅ Những gì đã học:
1. Kubernetes architecture (Control Plane, Nodes)
2. kubectl CLI basics
3. Namespaces và resource organization
4. Common kubectl commands

### 📚 Next Lab:
- **Lab 2**: Pods và Containers
- Tạo và quản lý Pods
- Multi-container pods
- Pod lifecycle

### 💡 Key Takeaways:

**Kubernetes:**
- Container orchestration platform
- Manages containerized applications
- Provides scaling, self-healing, service discovery

**Core Concepts:**
- Control Plane: Manages cluster
- Nodes: Run workloads
- Pods: Smallest deployable unit
- kubectl: CLI tool for interaction
